# MIMIC-II Treatment Effect Estimation with ehrapy

Here, we estimate treatment effects using survival analysis techniques.

## Case Study Data

This tutorial explores the MIMIC-II IAC dataset.
It was created for the purpose of a case study in the book: Secondary Analysis of Electronic Health Records, published by Springer in 2016.
In particular, the dataset was used throughout Chapter 16 (Data Analysis) by Raffa J. et al. to investigate the effectiveness of indwelling arterial catheters in hemodynamically stable patients with respiratory failure for mortality outcomes.
The dataset is derived from MIMIC-II, the publicly accessible critical care database.
It contains summary clinical data and  outcomes for 1,776 patients.

References:

[1] Critical Data, M.I.T., 2016. Secondary analysis of electronic health records (p. 244). Springer Nature. (https://link.springer.com/book/10.1007/978-3-319-43742-2)

[2] https://github.com/MIT-LCP/critical-data-book/tree/master/part_ii/chapter_16/jupyter

In [1]:
import ehrapy as ep
import ehrdata as ed
import pandas as pd

In [2]:
edata = ed.dt.mimic_2()

! File ehrapy_data/ehrapy_mimic2.csv already exists! Using already downloaded dataset...


In [3]:
ed.infer_feature_types(edata)
ed.replace_feature_types(edata, ["day_icu_intime_num", "hour_icu_intime"], "numeric")

! Features 'aline_flg', 'gender_num', 'service_num', 'day_icu_intime_num', 'hour_icu_intime', 'hosp_exp_flg', 'icu_exp_flg', 'day_28_flg', 'censor_flg', 'sepsis_flg', 'chf_flg', 'afib_flg', 'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg', 'mal_flg', 'resp_flg' were detected as categorical features stored numerically.Please verify and adjust if necessary using `ed.replace_feature_types`.


 Detected feature types for EHRData object with 1776 obs and 46 vars
╠══ 📅 Date features
╠══ 📐 Numerical features
║   ╠══ abg_count
║   ╠══ age
║   ╠══ bmi
║   ╠══ bun_first
║   ╠══ chloride_first
║   ╠══ creatinine_first
║   ╠══ hgb_first
║   ╠══ hospital_los_day
║   ╠══ hr_1st
║   ╠══ icu_los_day
║   ╠══ iv_day_1
║   ╠══ map_1st
║   ╠══ mort_day_censored
║   ╠══ pco2_first
║   ╠══ platelet_first
║   ╠══ po2_first
║   ╠══ potassium_first
║   ╠══ sapsi_first
║   ╠══ sodium_first
║   ╠══ sofa_first
║   ╠══ spo2_1st
║   ╠══ tco2_first
║   ╠══ temp_1st
║   ╠══ wbc_first
║   ╚══ weight_first
╚══ 🗂️ Categorical features
    ╠══ afib_flg (2 categories)
    ╠══ aline_flg (2 categories)
    ╠══ cad_flg (2 categories)
    ╠══ censor_flg (2 categories)
    ╠══ chf_flg (2 categories)
    ╠══ copd_flg (2 categories)
    ╠══ day_28_flg (2 categories)
    ╠══ day_icu_intime (7 categories)
    ╠══ day_icu_intime_num (7 categories)
    ╠══ gender_num (2 categories)
    ╠══ hosp_exp_flg (2 categories)
    ╠══ hour_icu_intime (24 categories)
    ╠══ icu_exp_flg (2 categories)
    ╠══ liver_flg (2 categories)
    ╠══ mal_flg (2 categories)
    ╠══ renal_flg (2 categories)
    ╠══ resp_flg (2 categories)
    ╠══ sepsis_flg (1 categories)
    ╠══ service_num (2 categories)
    ╠══ service_unit (3 categories)
    ╚══ stroke_flg (2 categories)

In [4]:
continuous_vars = edata.var.index[edata.var["feature_type"] == "numeric"]
categorical_vars = edata.var.index[edata.var["feature_type"] == "categorical"]

continuous_vars = continuous_vars.tolist()
categorical_vars = categorical_vars.tolist()
all_vars = continuous_vars + categorical_vars

## Case Study and Summary

Our Objective for this case study is as follows:
<br />
***
To estimate the effect that administration of IAC (Indwelling Arterial Catheter) during an ICU (Intensive Care Unit) admission has on 28 day
mortality in patients without sepsis who received mechanical ventilation within MIMIC II,
while adjusting for age, gender, severity of illness and comorbidities.
***
<br />
We will also not want to include the sepsis_flg variable as a covariate in any
of our models, as there are no patients with sepsis within this study to estimate the
effect of sepsis.

Usually, we will want to see how the population differs for different values of the covariate of interest.
In our case study, if the treated group (IAC) differed substantially from the untreated group (no IAC), then this may account for any effect we demonstrate.
We can do this by summarizing the two groups using the [tableone](https://github.com/tompollard/tableone) package.

In [5]:
import tableone

table = tableone.TableOne(edata.to_df(), all_vars, categorical=categorical_vars, groupby="aline_flg")
table.tabulate(tablefmt="html")

,,Missing,Overall,0,1
n,,,1776,792,984
"icu_los_day, mean (SD)",,0,3.3 (3.4),2.1 (1.9),4.3 (3.9)
"hospital_los_day, mean (SD)",,0,8.1 (8.2),5.4 (5.4),10.3 (9.3)
"age, mean (SD)",,0,54.4 (21.1),53.0 (21.7),55.5 (20.5)
"weight_first, mean (SD)",,110,80.1 (22.5),79.2 (22.6),80.7 (22.4)
"bmi, mean (SD)",,466,27.8 (8.2),28.0 (9.1),27.7 (7.5)
"sapsi_first, mean (SD)",,85,14.1 (4.1),12.7 (3.8),15.2 (4.0)
"sofa_first, mean (SD)",,6,5.8 (2.3),4.8 (2.1),6.6 (2.2)
"day_icu_intime_num, mean (SD)",,0,4.1 (2.0),4.0 (2.0),4.1 (2.0)
"hour_icu_intime, mean (SD)",,0,10.6 (7.9),9.9 (7.7),11.2 (8.1)


The IAC group differs in many respects to the non-IAC group.
Patients who were given IAC tended to have higher severity of illness at baseline (`sapsi_first` and `sofa_first`), slightly older, less likely to be from the MICU, and have slightly different comorbidity profiles when compared to the non-IAC group.

Next, we can see how the covariates are distributed among the different outcomes (death within 28 days versus alive at 28 days).
This will give us an idea of which covariates may be important for affecting the outcome.

In [6]:
table = tableone.TableOne(edata.to_df(), all_vars, categorical=categorical_vars, groupby="day_28_flg")
table.tabulate(tablefmt="html")

,,Missing,Overall,0,1
n,,,1776,1493,283
"icu_los_day, mean (SD)",,0,3.3 (3.4),3.2 (3.2),4.0 (4.0)
"hospital_los_day, mean (SD)",,0,8.1 (8.2),8.4 (8.4),6.4 (6.4)
"age, mean (SD)",,0,54.4 (21.1),50.8 (20.1),73.3 (15.3)
"weight_first, mean (SD)",,110,80.1 (22.5),81.4 (22.7),72.4 (19.9)
"bmi, mean (SD)",,466,27.8 (8.2),28.2 (8.3),26.0 (7.2)
"sapsi_first, mean (SD)",,85,14.1 (4.1),13.6 (3.9),17.3 (3.8)
"sofa_first, mean (SD)",,6,5.8 (2.3),5.7 (2.3),6.6 (2.4)
"day_icu_intime_num, mean (SD)",,0,4.1 (2.0),4.0 (2.0),4.1 (2.0)
"hour_icu_intime, mean (SD)",,0,10.6 (7.9),10.5 (7.9),11.0 (8.0)


Of the 984 subjects receiving IAC, 170 (17.2 %) died within 28 days, whereas 113 of 792 (14.2 %) died in the no-IAC group.
In a univariate analysis we can assess if the lower rate of mortality is statistically significant, by fitting a single covariate `aline_flg` logistic regression. 

To do this, we have to pay attention to our feature types.
The target of the GLM model has to be numerical (https://en.wikipedia.org/wiki/Generalized_linear_model), so we have to convert the `day_28_flg` to a numerical value. 

In [7]:
ep.ad.replace_feature_types(edata, ["day_28_flg"], "numeric")
glm_model = ep.tl.glm(
    edata,
    formula="day_28_flg ~ aline_flg",
    var_names=["day_28_flg", "aline_flg"],
    family="Binomial",
    use_feature_types=True,
)
res = glm_model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             day_28_flg   No. Observations:                 1776
Model:                            GLM   Df Residuals:                     1774
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -777.43
Date:                Fri, 05 Sep 2025   Deviance:                       1554.9
Time:                        17:16:58   Pearson chi2:                 1.78e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.001680
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.7932      0.102    -17.650      0.000      -1.992      -1.594
aline_flg[T.1]     0.2271      0.132      1.720      0.085      -0.032       0.486
==================================================================================
"""

Indeed, the p-value for `aline_flg` is about 0.09.
There are likely several important covariates that differed among those who received IAC and those who did not.
These may serve as confounders, and the possible association we observed in the univariate analysis may be stronger, nonexistent or in the opposite direction (i.e., IAC having lower rates of mortality) depending on the situation.
Our next step would be to adjust for these confounders.
A common approach is to fit all univariate models (one covariate at a time, as we did with `aline_flg`, but separately for each covariate and without `aline_flg`), and perform a hypothesis test on each model.
Any variables which had statistical significance under the univariate models would then be included in a multivariable model.
Another approach begins with the model we just fit (which only has `aline_flg` as a covariate), and then sequentially adds variables one at a time.
This approach is often called stepwise forward selection.

In our stepwise backwards elimination procedure, we are going to fit a full model and eliminate one variable at a time, until we are left with a model with only statistically significant covariates. 

Because `aline_flg` is the covariate of interest, it will remain in the model regardless of its statistical significance.
At each step, we need to define a criterion for eliminating variables.
One approach is to perform a hypothesis test for each covariate and remove the covariate with the largest p-value, unless all p-values are < 0.05 or the largest p-value corresponds to `aline_flg`.

Most covariates are binary or categorical, and we’ve already converted them to factors.
The disease severity scores (SAPS and SOFA) are continuous.
Adding them directly assumes a linear trend in mortality odds as scores change, which may not be appropriate.
To address this, we can discretize the scores into quantiles using [`cut2`](https://rdrr.io/cran/Hmisc/man/cut2.html).

We will apply this function to discretize the `sapsi_first` and `sofa_first` scores into quantiles.
These discretized scores can then be added to the survival model to better capture nonlinear relationships with mortality risk.

In [8]:
def cut2(edata: ed.EHRData, column_name: str, bins: int = 5):
    """Cut a continuous variable into bins and add it as a new categorical variable to the EHRData object."""
    created_col_name = f"{column_name}_{bins}_bins"
    if created_col_name in edata.obs.columns:
        return edata
    df = ed.io.to_pandas(edata)
    x = df[column_name]

    # Cut the data into bins
    bin_labels = [f"{column_name}_bin_{i + 1}" for i in range(bins)]
    binned = pd.qcut(x, q=bins, labels=bin_labels)
    binned.name = created_col_name

    df = pd.concat([df, binned], axis=1)

    new_obs = edata.obs.copy()
    new_obs = pd.concat([new_obs, binned], axis=1)

    new_var = edata.var.copy()
    new_var.loc[created_col_name] = "categorical"

    new_edata = ed.EHRData(
        X=df.values,
        obs=new_obs,
        var=new_var,
    )

    return new_edata

In [9]:
cut2_edata = cut2(edata, "sapsi_first")
cut2_edata = cut2(cut2_edata, "sofa_first")
cut2_edata.var.index

Index(['aline_flg', 'icu_los_day', 'hospital_los_day', 'age', 'gender_num',
       'weight_first', 'bmi', 'sapsi_first', 'sofa_first', 'service_unit',
       'service_num', 'day_icu_intime', 'day_icu_intime_num',
       'hour_icu_intime', 'hosp_exp_flg', 'icu_exp_flg', 'day_28_flg',
       'mort_day_censored', 'censor_flg', 'sepsis_flg', 'chf_flg', 'afib_flg',
       'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg',
       'mal_flg', 'resp_flg', 'map_1st', 'hr_1st', 'temp_1st', 'spo2_1st',
       'abg_count', 'wbc_first', 'hgb_first', 'platelet_first', 'sodium_first',
       'potassium_first', 'tco2_first', 'chloride_first', 'bun_first',
       'creatinine_first', 'po2_first', 'pco2_first', 'iv_day_1',
       'sapsi_first_5_bins', 'sofa_first_5_bins'],
      dtype='object')

We added a new column in the EHRData object for each of the `sapsi_first` and `sofa_first` bins.
Now every observation has a value for `sapsi_first_bin` and `sofa_first_bin` which is the quartile that the observation falls into.
We can now add these to the model, instead of the continuous `sapsi_first` and `sofa_first` scores.

In [10]:
dependent_var = "day_28_flg"
not_dependent_vars = [
    "aline_flg",
    "age",
    "gender_num",
    "sapsi_first_5_bins",
    "sofa_first_5_bins",
    "service_unit",
    "chf_flg",
    "afib_flg",
    "renal_flg",
    "liver_flg",
    "copd_flg",
    "cad_flg",
    "stroke_flg",
    "mal_flg",
    "resp_flg",
]
formula = f"{dependent_var} ~ {' + '.join(not_dependent_vars)}"
var_names = not_dependent_vars + [dependent_var]
full_model = ep.tl.glm(
    cut2_edata,
    var_names,
    formula,
    missing="drop",
    family="Binomial",
    use_feature_types=True,
)
full_model_result = full_model.fit()
full_model_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             day_28_flg   No. Observations:                 1684
Model:                            GLM   Df Residuals:                     1661
Model Family:                Binomial   Df Model:                           22
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -479.06
Date:                Fri, 05 Sep 2025   Deviance:                       958.12
Time:                        17:16:58   Pearson chi2:                 1.35e+03
No. Iterations:                     7   Pseudo R-squ. (CS):             0.2311
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                  -7.4832      0.849     -8.809      0.000      -9.148      -5.818
aline_flg[T.1]                              0.0327      0.204      0.160      0.873      -0.367       0.433
gender_num[T.1.0]                           0.1422      0.172      0.824      0.410      -0.196       0.480
sapsi_first_5_bins[T.sapsi_first_bin_2]     0.0390      0.342      0.114      0.909      -0.631       0.709
sapsi_first_5_bins[T.sapsi_first_bin_3]     0.6417      0.293      2.190      0.028       0.067       1.216
sapsi_first_5_bins[T.sapsi_first_bin_4]     0.5174      0.293      1.769      0.077      -0.056       1.091
sapsi_first_5_bins[T.sapsi_first_bin_5]     1.3695      0.295      4.637      0.000       0.791       1.948
sofa_first_5_bins[T.sofa_first_bin_2]       0.5223      0.302      1.729      0.084      -0.070       1.114
sofa_first_5_bins[T.sofa_first_bin_3]       0.6462      0.301      2.144      0.032       0.055       1.237
sofa_first_5_bins[T.sofa_first_bin_4]       0.7943      0.290      2.735      0.006       0.225       1.363
sofa_first_5_bins[T.sofa_first_bin_5]       0.8867      0.336      2.639      0.008       0.228       1.545
service_unit[T.MICU]                        1.0609      0.681      1.559      0.119      -0.273       2.395
service_unit[T.SICU]                        0.6666      0.674      0.990      0.322      -0.654       1.987
chf_flg[T.1]                                0.2347      0.234      1.005      0.315      -0.223       0.693
afib_flg[T.1]                               0.5544      0.211      2.628      0.009       0.141       0.968
renal_flg[T.1]                             -0.7834      0.409     -1.917      0.055      -1.584       0.018
liver_flg[T.1]                              0.4643      0.341      1.360      0.174      -0.205       1.133
copd_flg[T.1]                               0.2533      0.244      1.037      0.300      -0.225       0.732
cad_flg[T.1]                               -0.2444      0.288     -0.849      0.396      -0.808       0.319
stroke_flg[T.1]                             2.0563      0.220      9.348      0.000       1.625       2.487
mal_flg[T.1]                                0.4879      0.209      2.334      0.020       0.078       0.898
resp_flg[T.1]                               0.7172      0.192      3.735      0.000       0.341       1.094
age                                         0.0425      0.006      6.841      0.000       0.030       0.055
===========================================================================================================
"""

Some of the covariates are statistically, while others may be expendable.
Ideally, we would like as simple of a model as possible that can explain as much of the variation in the outcome as possible.
We will attempt to remove our first covariate by the procedure we outlined above.
For each of the variables we consider removing, we could fit a logistic regression model without that covariate, and then test it against the current model.

The `drop1` function will take the data, the dependent variable and a list of independent variables and return the p-value for each variable. We can then choose to remove the variable with the largest p-value, this process would then be repeated until all p-values are less than 0.05. As this is an example, we will only do one iteration.

In [11]:
from scipy.stats import chi2


def drop1(
    edata: ed.EHRData,
    dependent_var: str,
    not_dependent_vars: list[str],
    missing: str = "drop",
):
    """Python implementation of R's drop1 function using ehrapy.tl.glm.

    Args:
        edata: The EHRData object for the OLS model.
        dependent_var: The name of the dependent variable.
        independent_vars: A list of independent variables.
        missing: How to handle missing data. Default is "drop".

    Returns:
        pd.DataFrame: A DataFrame containing AIC, BIC, and LRT p-values for each model with one term dropped.
    """
    # Fit the full model
    formula = f"{dependent_var} ~ {' + '.join(not_dependent_vars)}"
    var_names = not_dependent_vars + [dependent_var]
    full_model = ep.tl.glm(
        edata,
        var_names=var_names,
        formula=formula,
        missing=missing,
        family="Binomial",
        use_feature_types=True,
    )
    full_model_result = full_model.fit()

    results = []

    # Drop one term at a time and compare models
    for term in not_dependent_vars:
        reduced_formula = (
            f"{formula.split('~')[0].strip()} ~ {' + '.join([t for t in not_dependent_vars if t != term])}"
        )
        reduced_model = ep.tl.glm(
            edata,
            var_names=var_names,
            formula=reduced_formula,
            missing=missing,
            family="Binomial",
            use_feature_types=True,
        )
        reduced_model_result = reduced_model.fit()

        df = full_model.df_model - reduced_model.df_model
        aic = reduced_model_result.aic
        deviance = reduced_model_result.deviance

        lrt_stat = -2 * (reduced_model_result.llf - full_model_result.llf)
        lrt_pvalue = chi2.sf(lrt_stat, df=df)  # p-value using chi-squared distribution

        results.append(
            {
                "Dropped Term": term,
                "Deviance": deviance,
                "AIC": aic,
                "LRT Statistic": lrt_stat,
                "LRT p-value": lrt_pvalue,
            }
        )

    results_df = pd.DataFrame(results)
    return results_df

In [12]:
drop1(cut2_edata, dependent_var, not_dependent_vars, missing="drop")

,Dropped Term,Deviance,AIC,LRT Statistic,LRT p-value
0,aline_flg,958.146491,1002.146491,0.025627,8.728142e-01
1,age,1010.700508,1054.700508,52.579644,4.131493e-13
2,gender_num,958.832055,1002.832055,0.711191,3.990487e-01
3,sapsi_first_5_bins,1069.795683,1107.795683,111.674819,3.197094e-23
4,sofa_first_5_bins,978.050947,1016.050947,19.930083,5.155226e-04
5,service_unit,963.069661,1005.069661,4.948797,8.421362e-02
6,chf_flg,959.122961,1003.122961,1.002098,3.168034e-01
7,afib_flg,964.939191,1008.939191,6.818327,9.022704e-03
8,renal_flg,962.106642,1006.106642,3.985778,4.588591e-02
9,liver_flg,959.886794,1003.886794,1.765931,1.838865e-01


`aline_flg` has the largest p-value, but we stipulated in our model selection plan that we would retain this covariate as it’s our covariate of interest.
We will then go to the next largest p-value which is the `gender_num` variable.
From here we can update our model, and repeat the backwards elimination step on the updated model.

In [13]:
reduced_vars = [ind_var for ind_var in not_dependent_vars if ind_var != "gender_num"]
drop1(cut2_edata, dependent_var, reduced_vars, missing="drop")

,Dropped Term,Deviance,AIC,LRT Statistic,LRT p-value
0,aline_flg,958.853545,1000.853545,0.021491,8.834499e-01
1,age,1010.759880,1052.759880,51.927825,5.757865e-13
2,sapsi_first_5_bins,1070.187660,1106.187660,111.355606,3.739818e-23
3,sofa_first_5_bins,979.692198,1015.692198,20.860144,3.375408e-04
4,service_unit,963.599162,1003.599162,4.767108,9.222224e-02
5,chf_flg,959.775853,1001.775853,0.943799,3.313029e-01
6,afib_flg,965.669262,1007.669262,6.837207,8.927832e-03
7,renal_flg,962.629266,1004.629266,3.797212,5.133801e-02
8,liver_flg,960.558143,1002.558143,1.726089,1.889112e-01
9,copd_flg,959.933978,1001.933978,1.101923,2.938444e-01


## Conclusion

We see that `aline_flg` still has the largest p-value, but `cad_flg` has the second largest, so we would choose to remove it next. 